In [1]:
%load_ext autoreload
%autoreload 2
import manenv
from manenv.utils.product_utils import *
from manenv.core.world import World
from manenv.core.component import *
from manenv.render_wrapper import RenderWrapper
from manenv.core.product import *
from manenv.core.effector import *

import manenv.effectors as eff
import manenv.components as cmp


pygame-ce 2.4.1 (SDL 2.28.5, Python 3.11.9)


In [2]:
world = World((3, 3))
s = np.zeros((5, 5), dtype=np.int8)
s[1][2] = 2
s[1][1] = 2
s[2][1] = 1
s[3][1] = 1
product = Product(s)
world.place_component(make_vector(1, 1), cmp.Conveyor(
    np.array([
    [0, 0, 0],
    [1, 0, -1],
    [0, 0, 0]
])))
world.place_component(make_vector(2, 0), cmp.Conveyor(
    np.array([
    [0, -1, 0],
    [0, 0, 1],
    [0, 0, 0] 
])))
world.place_component(make_vector(2, 1), cmp.Outport())

grabber = eff.Grabber(make_vector(0, 2))
buddy = eff.Grabber(make_vector(1, 3))
welder = eff.Welder(make_vector(1, 2))
transformer = eff.Transformer(1, 2, make_vector(2,  4))
discard = eff.Acceptor(make_vector(1, 7))

world.place_component(make_vector(1, 2), cmp.Spawner(product))
assembler : cmp.Assembler = cmp.Assembler(make_vector(10, 10), [grabber, buddy, welder, discard])
world.place_component(make_vector(1, 0), assembler)
wrapper = RenderWrapper(world)

In [3]:
# wrapper.reset()
# assembler.place_in_inventory(product.copy())
# assembler.place_in_inventory(product.copy())
# world.update()

# grabber.set_action(eff.GrabberActions.GRAB_INVENTORY.value)
# grabber.set_action(eff.GrabberActions.GRAB_INVENTORY.value)
# buddy.set_action(eff.GrabberActions.GRAB_INVENTORY.value)
# world.update()

# grabber.set_action(eff.GrabberActions.RELEASE.value)
# buddy.set_action(eff.GrabberActions.RELEASE.value)
# world.update()

# welder.set_action(eff.WelderActions.WELD_EAST.value)
# world.update()
# world.update()

# grabber.set_action(eff.GrabberActions.GRAB.value)
# world.update()

# grabber.set_action(eff.GrabberActions.MOVE_RIGHT.value)
# world.update()

# grabber.set_action(eff.GrabberActions.RELEASE.value)
# world.update()

# transformer.set_action(eff.TransformerActions.TRANSFORM.value)
# world.update()
# world.update()

# grabber.set_action(eff.GrabberActions.MOVE_RIGHT.value)
# world.update()
# world.update()

# grabber.set_action(eff.GrabberActions.MOVE_RIGHT)
# world.update()

# grabber.set_action(eff.GrabberActions.MOVE_RIGHT)
# world.update()


In [4]:
# wrapper.render()

# Gym Environment Test

In [5]:
from manenv.gym_wrapper import MARLFactoryEnvironment

In [6]:
world.reset()
environment = MARLFactoryEnvironment(world)
renderer = RenderWrapper(environment._world)

In [7]:
from pettingzoo.test import parallel_api_test

In [8]:
def test():
    parallel_api_test(environment, num_cycles=1_000_000)

In [9]:
import cProfile
cProfile.run('test()', sort = 'time')

c:\Users\Joaquin\miniconda3\envs\thesis\Lib\site-packages\pettingzoo\test\parallel_test.py:36: UserWarning: This environment does not have possible_agents defined.
This is not a required part 'of the API as environments with procedurally
generated agents cannot always have this property defined. However, this is
very uncommon and these features should be included whenever possible as all
standard learning code requires these properties. Also not that if you do not
have possible_agents it should also not be possible for you to expose the possible_agents
list and observation_spaces, action_spaces dictionaries.
  warnings.warn(missing_attr_warning.format(name="possible_agents"))
c:\Users\Joaquin\miniconda3\envs\thesis\Lib\site-packages\pettingzoo\test\parallel_test.py:88: UserWarning: Live agent was not given reward
  warnings.warn(f"Live agent was not given {k}")


         7963249 function calls (7963247 primitive calls) in 31.942 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
  1060090   27.807    0.000   29.780    0.000 world.py:67(remove_product)
   529420    1.921    0.000    1.921    0.000 {method 'remove' of 'list' objects}
     2263    0.666    0.000   31.149    0.014 conveyor.py:55(update)
  1061220    0.534    0.000    0.632    0.000 world.py:55(place_product)
  2099949    0.194    0.000    0.194    0.000 world.py:60(is_product_placed)
   527161    0.139    0.000    0.159    0.000 inventory.py:20(add_product)
     1132    0.138    0.000   31.835    0.028 world.py:125(update)
     1131    0.083    0.000    0.243    0.000 outport.py:14(update)
  1065604    0.068    0.000    0.068    0.000 {method 'append' of 'list' objects}
    20966    0.035    0.000    0.035    0.000 _methods.py:90(_clip)
   530664    0.031    0.000    0.031    0.000 {method 'add' of 'set' objects}
   55876

KeyboardInterrupt: 

In [ ]:
for _ in range(100):
    action = environment._action_space.sample()
    environment.step(action)
renderer.render()

KeyError: 12